`Дисциплина: Методы и технологии машинного обучения` <br>
`Уровень подготовки: бакалавриат`<br>
`Направление подготовки: 01.03.02 Прикладная математика и информатика`<br>
`Семестр: осень 2021/2022` <br>
# Лабораторная работа №6: Машины опорных векторов.

Кечин Дмитрий, 10 вариант.



1.  В практических примерах ниже показано:
2.  Как использовать конвейеры для подгонки модели и применения её к новым данным

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.


## Данные



1.   `fixed_acidity` – постоянная кислотность;
2.   `volatile_acidity` – переменная кислотность;
3.   `citric_acid` – содержание лимонной кислоты;
4.   `residual_sugar` - остаточный сахар;
5.   `chlorides` – содержание хлоридов;
6.   `free_sulfur_dioxide` – содержание диоксида серы в свободном виде;
7.   `total_sulfur_dioxide` – общее содержание диоксида серы;
8.   `density` – плотность;
9.   `pH` – кислотность;
10.  `sulphates` – содержание сульфатов;
11.  `alcohol` – содержание алкоголя;
12.  `quality` – балльная оценка качества вина, от 0 до 10;
13.  `Y` – целевая переменная: 1 = высокое качество (quality > 5), 0 = низкое (quality <= 5).

# Загружаем пакеты

In [58]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency, t
#  для таймера
import time

# загрузка пакетов: модели ------------------------------------------------- 
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression 
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split 
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report

# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets

# загрузка пакетов: модели -------------------------------------------------
#  дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree


# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier

# для теста хи-квадрат
import scipy

In [59]:
# константы
#  ядро для генератора случайных чисел
my_seed = 10
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [60]:
# функция форматирования результатов с использованием Markdown
def printmd(string):
    display(Markdown(string))
    
# функции для попарной конкатенации элементов двух списков
concat_func_md = lambda x, y: '`' + str(x) + "`:&ensp;&ensp;&ensp;&ensp;" + str(y)
concat_func = lambda x, y: str(x) + ' ' * 4 + str(y)


# функция, которая строит график важности признаков в модели случайного леса
#  источник: https://www.analyseup.com/learn-python-for-data-science/python-random-forest-feature-importance-plot.html
def plot_feature_importance(importance, names, model_type) :
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,
                      inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title('Важность признаков в модели: ' + model_type)
    plt.xlabel('Важность признака')
    plt.ylabel('')

In [61]:
# загружаем таблицу и превращаем её во фрейм
DF_raw = pd.read_csv('https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/winequality-red_for_lab.csv')

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (1599, 13)


In [62]:
DF_raw.iloc[:].head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Y
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [63]:
DF_raw = DF_raw.drop(["quality"], axis = 1)

In [64]:
# типы столбцов
DF_raw.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
Y                         int64
dtype: object

In [65]:
DF_raw.isna().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
Y                       0
dtype: int64

In [66]:
DF = DF_raw.sample(frac = 0.85, random_state = my_seed)
DF_predict = DF_raw.drop(DF.index)

In [67]:
DF.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
count,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000,1359.000000
mean,8.282561,0.528503,0.268506,2.501288,0.087788,16.063282,47.189845,0.996716,3.313297,0.659007,10.410755,0.524650
std,1.716960,0.178512,0.193256,1.370224,0.047647,10.591927,33.564421,0.001873,0.156033,0.171438,1.052374,0.499576
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,0.000000
25%,7.100000,0.400000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,0.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,14.000000,38.000000,0.996720,3.310000,0.620000,10.100000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,22.000000,63.500000,0.997800,3.405000,0.730000,11.100000,1.000000
max,15.600000,1.580000,1.000000,15.400000,0.610000,72.000000,289.000000,1.003690,4.010000,2.000000,14.000000,1.000000


# SVM

В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент

In [68]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 0:14].values)

# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
    np.around(frac_var_expl, 3),
    '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.258 0.187 0.139 0.101 0.082 0.055 0.052 0.045 0.035 0.028 0.015 0.005] 
Общая сумма долей: 1.0


Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline и оценим точность логистической регрессии с помощью перекрёстной проверки

# Модель логистической регрессии с перекрёстной проверкой

In [69]:
# данные для обучения моделей
X_train = DF.iloc[:, 0:12] 
y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.834


In [70]:
X_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
1518,7.4,0.470,0.46,2.2,0.114,7.0,20.0,0.99647,3.32,0.63,10.5,0
1246,7.4,0.740,0.07,1.7,0.086,15.0,48.0,0.99502,3.12,0.48,10.0,0
544,14.3,0.310,0.74,1.8,0.075,6.0,15.0,1.00080,2.86,0.79,8.4,1
1343,7.5,0.510,0.02,1.7,0.084,13.0,31.0,0.99538,3.36,0.54,10.5,1
428,9.1,0.520,0.33,1.3,0.070,9.0,30.0,0.99780,3.24,0.60,9.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1169,7.6,0.500,0.29,2.3,0.086,5.0,14.0,0.99502,3.32,0.62,11.5,1
1228,5.1,0.420,0.00,1.8,0.044,18.0,88.0,0.99157,3.68,0.73,13.6,1
515,8.5,0.655,0.49,6.1,0.122,34.0,151.0,1.00100,3.31,1.14,9.3,0
826,7.5,0.270,0.34,2.3,0.050,4.0,8.0,0.99510,3.40,0.64,11.0,1


In [71]:
y_train


1518    0
1246    0
544     1
1343    1
428     0
       ..
1169    1
1228    1
515     0
826     1
1340    1
Name: Y, Length: 1359, dtype: int64

X_train и y_train удовлетворяют условию.

# SVM с перекрёстной проверкой

In [72]:
pipe_svc = make_pipeline(StandardScaler(), 
                         SVC(random_state = my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1] 
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, 
                  scoring = 'accuracy', refit = True, cv = kfold, 
                  n_jobs = -1)

# таймер
tic = time.perf_counter()

# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)

# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 13.86 секунд


In [73]:
# точность лучшей модели
np.around(gs.best_score_, 3)

1.0

Точность > 96%, изменялся лист param_range.

In [74]:
# параметры лучшей модели
# * ядерная функция 
gs.best_estimator_.get_params()['svc__kernel']

'linear'

In [75]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

0.001

In [76]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

'scale'

In [77]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

3

In [78]:
score.append(np.around(gs.best_score_,3))
score_models.append('sc_pca_svc')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


# **Метод kNN**
Реализуем метод k-ближайших соседей с преобразованием PCA.

Загрузим доп пакеты

In [79]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [80]:
k_range = list(range(1,32))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 5)), ('knn', KNeighborsClassifier())])
param_grid = {
    'knn__n_neighbors': list(range(1,32))
}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 4.46 секунд


В ходе обучения модели я менял такие параметры как: n_splits (не дало особого увеличения точности) и n_components в PCA (дало прибавку к точности с 0.834 до 0.904)

In [81]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


# **Прогноз на отложенные наблюдения по лучшей модели**

Ещё раз посмотрим на точность построенных моделей.

In [82]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,sc_pca_logit,0.834
1,sc_pca_svc,1.000
2,sc_pca_knn,0.904


Все модели показывают хорошую точность по показателю $Acc$, при этом самой точной оказывается модель PCA + SVC. Сделаем прогноз на отложенные наблюдения.

In [83]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat=gs.best_estimator_.predict(X=DF_predict.iloc[:, 0:12])
# точность# характеристики точности
print(classification_report(DF_predict.iloc[:,-1],y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        98
           1       1.00      1.00      1.00       142

    accuracy                           1.00       240
   macro avg       1.00      1.00      1.00       240
weighted avg       1.00      1.00      1.00       240

